In [1]:
import os
import sys

import numpy as np
import pandas as pd
import itertools

#plotting tools
import matplotlib as mpl
import seaborn as sn
from plotnine import *

#Suppress pivot warning when savining plots with plotnine
import warnings
warnings.simplefilter("ignore")

%load_ext watermark

In [2]:
pwd = os.path.abspath('../')
sys.path.append(os.path.join(pwd, 'code/')) 
from utilities import *

In [3]:
#Set some plotting defaults
sn.set_style('ticks')
mpl.rcParams.update({'text.color': 'black', 'axes.labelcolor': 'black', 
                     'xtick.color': 'black', 'ytick.color': 'black',  'xtick.labelsize':12,  'ytick.labelsize':12,
                     'axes.labelsize':12,  'axes.titlesize':12 , 'xtick.labelsize':12 , 'ytick.labelsize':12, 'figure.dpi':100, 'savefig.dpi':300,
                     'savefig.bbox': 'tight', 'savefig.transparent': True, 'mathtext.default':'regular'})
sn.set_context('notebook')
%matplotlib inline
%config InlineBackend.figure_format='retina'

Plot dimer concentrations for the simulations of single layer dimerization networks with a single input titration (i.e. $x \in R$). For the simulations, $K$ values were sampled from the range  ($10^{-6}$, $10^6$) and accessory monomer levels ($a$) from the range ($10^{-3}$, $10^3$). 


In [20]:
#Using plotnine to make facet plots. Tried using Seaborn but found plotnine to be simpler. 
def plot_curves_facet_univ(simfile, input_titration, m = 3, 
                           outputweights = None, 
                           n_univ = 100, xscale = 'log', 
                           univset = None, ncol = 10, 
                           wadjust = 0.25, norm = 'max', 
                           save = False, outfileprefix = ''):
    """
    Plot output curves for 1-input dimer network simulations.
    
    Parameters
    ----------
    simfile : string
        path to .npy file containing results of the network simulation
        Loaded into S_all
    input_titration : array_like. shape (n_titrations,)
        initial concentration of the input monomer species. 
    m : int. Default 3
        Number of monomer species. Default is 3. 
    outputweights : array_like. shape (n_titrations, n). Default is None. 
        output for each network corresponds to np.matmul(S, outputweights) 
        where the matrix S contains the steady state concentration of all species (monomers & dimers)
        for a given parameter universe. If None, then outputweights is assigned the identity matrix.
    n_univ : int. Default 100
        Number of parameter universes to plot in the figure. 
        Each parameter universes corresponds to a row in the figure. 
        Should be ≤ S_all.shape[2]
    xscale : string. Default 'log'
        scale for x-axis. Should be 'log' or 'linear'
    univset : array_like. shape (n_univ, ). Default None
        index of the parameter universes to plot. 
        if None then univset = np.arange(n_univ)
    ncol : int. Default 10
        Number columns for facetting the plots. 
    wadjust : float. Default 0.25
        horizontal spacing between subplots.  
    norm : string. Default 'max'. Options are 'max', 'scale' or 'none'
        Indicates how to normalize the output concentrations. 
    save : Bool. Default False
        If True, save plot figure
    outfileprefix : string. Default ''
        relative path for saving figure. 
    Returns
    -------
    plot_list : list. Shape (outputweights.shape[1],)
        Each element corresponds to a plotnine figure object for a single output.
    """
    S_all = np.load(simfile)
    species_names = np.array(make_nXn_species_names(m))
    
    if univset is None:
        univset = np.arange(n_univ)
    else:
        n_univ = len(univset)
        
    S_subset = S_all[:,:,univset]
    
    if outputweights is None:
        outputweights = np.diag(np.ones(len(species_names)))
        out = np.moveaxis(S_subset, 2, 0)
    else:
        out = np.matmul(np.moveaxis(S_subset, 2, 0), outputweights)
    
    plot_list = [0] * outputweights.shape[1]
    for i in range(outputweights.shape[1]):
        out_name = species_names[outputweights[:,i].astype(bool)]
        out_df = pd.DataFrame({'x': np.tile(input_titration, n_univ), 
                               'univ': np.repeat(univset, input_titration.shape[0]), 
                               'out': out[:,:,i].reshape(input_titration.shape[0]*n_univ, 1).squeeze()})
        if norm == 'max':
            out_df['out_norm'] = out_df.groupby('univ', group_keys=False).apply(lambda x: x.out/x.out.max())
        elif norm == 'scale':
            out_df['out_norm'] = out_df.groupby('univ', group_keys=False).apply(lambda x: x.out-x.out.min())
            out_df['out_norm'] = out_df.groupby('univ', group_keys=False).apply(lambda x: x.out_norm/x.out_norm.max())
        elif norm == 'none':
            out_df['out_norm'] = out_df.out
        else:
            raise ValueError('Unexpected value for norm. Please choose from ["max", "scale", "none"]')
        
        p = (ggplot(out_df, aes('x', 'out_norm'))
             + geom_line(size = 0.5)
             + facet_wrap('~univ', ncol = ncol)
             + theme_classic()
             + ylab(f'$[{"_".join(out_name)}]_{{eq}}$')
             + scale_y_continuous(breaks = [0,1])
             + theme(strip_background=element_blank(), 
                 panel_border=element_blank(),
                 axis_line = element_line(size =0.5),
                 axis_ticks = element_line(size = 0.5),
                 axis_ticks_minor_x=element_blank(),
                 legend_position='none', aspect_ratio=0.7,
                 text=element_text(family='Helvetica', color='black'))
            )
        if xscale == 'log':
            maxmin1 = [input_titration.min(), 1, input_titration.max()]
            p = p + scale_x_log10(breaks = maxmin1,
                                 labels = [f'{int(np.log10(i))}' for i in maxmin1])
            p = p + xlab('$log_{10}[M1]_{tot}$')
        elif xscale == 'linear':
            maxminmid = np.linspace(input_titration.min(), input_titration.max(), 3)
            p = p + scale_x_continuous(breaks=maxminmid) + theme(axis_text_x = element_text(angle = 45))
            p = p + xlab('$[M1]_{tot}$')
        else:
            raise ValueError('Unexpected value for xscale. Please choose "log" or "linear"')
    
        if wadjust is not None:
            p = p + theme(subplots_adjust={'wspace': wadjust})
            
        plot_list[i] = p
        
        if save:
            p.save(f'{outfileprefix}{"_".join(out_name)}eq_curves_{n_univ}univ_{m}M_{norm}Norm_{xscale}Xscale.pdf', dpi=300)
    
    return plot_list


Randomly select 100 different simulations (out of the 1000) to plot. 

In [26]:
rng = np.random.default_rng(42)
univ_set = rng.choice(1000, size =100, replace = False)
np.save('../plots/20220420_1input_randomParams/plotted_univ_set_1K.npy', univ_set)

In [22]:
outfileprefix = '../plots/20220420_1input_randomParams/'
if not os.path.isdir(outfileprefix):
    os.mkdir(outfileprefix)

In [23]:
#Load input titration. Should be the same for all network sizes
C0 = np.load('../data/20220420_1input_randomParams/C0_3M_1000k.npy')
input_titration = C0[:,0]



In [24]:
_ = plot_curves_facet_univ('../data/20220420_1input_randomParams/S_all_3M_1000k.npy', 
                           input_titration, m = 3, 
                           outputweights = None, univset = univ_set, xscale = 'log',
                           ncol = 10, norm = 'max', save = True, outfileprefix = outfileprefix)


Plot results for larger networks (m = 4 - 6)

In [13]:
_ = plot_curves_facet_univ('../data/20220420_1input_randomParams/S_all_4M_1000k.npy', 
                           input_titration, m = 4, 
                           outputweights = None, univset = univ_set, xscale = 'log',
                           ncol = 10, norm = 'max', save = True, outfileprefix = outfileprefix)

In [14]:
_ = plot_curves_facet_univ('../data/20220420_1input_randomParams/S_all_5M_1000k.npy', 
                           input_titration, m = 5, 
                           outprefix = outfileprefix, 
                           outputweights = None, univset = univ_set, xscale = 'log',
                           ncol = 10, norm = 'max', save = True, outfileprefix = outfileprefix)

In [15]:
_ = plot_curves_facet_univ('../data/20220420_1input_randomParams/S_all_6M_1000k.npy', 
                           input_titration, m = 6, 
                           outprefix = outfileprefix, 
                           outputweights = None, univset = univ_set, xscale = 'log',
                           ncol = 10, norm = 'max', save = True, outfileprefix = outfileprefix)

Now plot results from simulation with single parameter titrations. 
On each plot, show separate curves for the different values of the titrated parameter. 


In [12]:
#Using plotnine to make facet plots. Tried using Seaborn but found plotnine to be cleaner and easier. 
def plot_curves_facet_univXparam(simfile, input_titration, m = 3, n_input = 1, 
                                 outputweights = None, n_univ = 100, xscale = 'log', palette = None,
                                 univset = None, norm = 'max', fontsize = 12, linesize = 0.5,
                                 xbreaks = None, xlabels = None, wadjust= None, save = False, outfileprefix = ''):
    """
    Plot output curves for 1-input dimer network simulations with single parameter titrations.
    
    Parameters
    ----------
    simfile : string
        path to .npy file containing results of the network simulation
        Loaded into S_all
    input_titration : array_like. shape (n_titrations,)
        initial concentration of the input monomer species. 
    m : int. Default 3
        Number of monomer species. Default is 3. 
    n_input : int. Default 1
        Number of input monomer species. 
    outputweights : array_like. shape (n_titrations, n). Default is None. 
        output for each network corresponds to np.matmul(S, outputweights) 
        where the matrix S contains the steady state concentration of all species (monomers & dimers)
        for a given parameter universe. If None, then outputweights is assigned the identity matrix.
    n_univ : int. Default 100
        Number of parameter universes to plot in the figure. 
        Each parameter universes corresponds to a row in the figure. 
        Should be ≤ S_all.shape[2]
    xscale : string. Default 'log'
        scale for x-axis. Should be 'log' or 'linear'
    palette : None or list of hexcode colors. Default None. 
        Color palette for output curves. 
        If None, uses Seaborn's "rocket_r" palette. 
    univset : array_like. shape (n_univ, ). Default None
        index of the parameter universes to plot. 
        if None then univset = np.arange(n_univ)
    norm : string. Default 'max'. Options are 'max', 'scale' or 'none'
        Indicates how to normalize the output concentrations. 
    fontsize : int. Default 12
        font size for text
    linesize : int. Default 0.5
        size for curve thickness. 
    xbreaks : array_like. Numeric elements. Default None
        breakpoints for x-axis
        If None, defaults to 3 evenly spaced values between min and max of x-axis
    xlabels : array_like. String or numeric elements. Default None. 
        labels for x-axis
        If None, defaults to xbreaks. 
    wadjust : float. Default 0.25
        horizontal spacing between subplots.  
    save : Bool. Default False
        If True, save plot figure
    outfileprefix : string. Default ''
        relative path for saving figure. 
        
    Returns
    -------
    plot_list : list. Shape (outputweights.shape[1],)
        Each element corresponds to a plotnine figure object for a single output.
    out_df_list : List. Shape (outputweights.shape[1], )
        Each element corresponds to a pandas dataframe used for plotting a single output. 
        Columns of each dataframe include ['x', 'out', 'univ', 'param', 'param_level', 'out_norm']
    """
    S_all = np.load(simfile)
    species_names = np.array(make_nXn_species_names(m))
    params = make_Kij_names(n_input=n_input, n_accesory=(m-n_input)) 
    params += [f'M{i}' for i in range(n_input+1, m+1)]

    n_titrations = len(input_titration)
    n_params = S_all.shape[3]
    n_param_titration = S_all.shape[4]
    
    if palette is None:
         palette = sn.color_palette("rocket_r", n_param_titration).as_hex()
            
    if univset is None:
        univset = np.arange(n_univ)
    else:
        n_univ = len(univset)
    
    S_subset = S_all[:,:,univset,:,:]
    S_subset = np.moveaxis(S_subset, [0, 1], [3,4])
    
    if outputweights is None:
        outputweights = np.diag(np.ones(len(species_names)))
        out = S_subset.copy()
    else:
        out = np.matmul(S_subset, outputweights)
    
    out = out.reshape(-1, outputweights.shape[1])
    
    #List to store ggplot objects
    plot_list = [0] * outputweights.shape[1]
    out_df_list = [0] * outputweights.shape[1]
    
    #Columns for data frame consistent across all output species
    input_titration_list =  np.tile(input_titration, n_univ*n_params*n_param_titration)
    univ_list = np.repeat(univset, n_titrations*n_params*n_param_titration)
    param_list = np.tile(np.repeat(params, n_titrations*n_param_titration), n_univ)
    param_levels_list = np.tile(np.repeat(np.arange(n_param_titration), n_titrations), n_univ*n_params)
    
    for i in range(outputweights.shape[1]):
        out_name = species_names[outputweights[:,i].astype(bool)]
        out_df = pd.DataFrame({'x': input_titration_list,
                     'out': out[:,i],
                     'univ': univ_list,
                     'param': param_list,
                     'param_level': param_levels_list})
        
        if norm == 'max':
            out_df['out_norm'] = out_df.groupby(['univ', 'param', 'param_level'], group_keys=False).apply(lambda x: x.out/x.out.max())
        elif norm == 'scale':
            out_df['out_norm'] = out_df.groupby(['univ', 'param', 'param_level'], group_keys=False).apply(lambda x: x.out-x.out.min())
            out_df['out_norm'] = out_df.groupby(['univ', 'param', 'param_level'], group_keys=False).apply(lambda x: x.out_norm/x.out_norm.max())
        elif norm == 'none':
            out_df['out_norm'] = out_df.out
        else:
            raise ValueError('Unexpected value for norm. Please choose from ["max", "scale", "none"]')
        
        p = (ggplot(out_df, aes('x', 'out_norm', color='factor(param_level)'))
             + geom_line(size = linesize)
             + facet_grid('univ~param')
             + theme_classic()
             + ylab(f'$[{"_".join(out_name)}]_{{eq}}$')
             + scale_y_continuous(breaks = [0,1])
             + scale_color_manual(palette.as_hex())
             + theme(strip_background=element_blank(), 
                     panel_border=element_blank(),
                     axis_line = element_line(size = linesize),
                     axis_ticks = element_line(size = linesize),
                     axis_ticks_length= 1,
                     axis_ticks_minor_x=element_blank(),
                     legend_position='none', aspect_ratio=0.7,
                     text=element_text(family='Helvetica', color='black', size=fontsize))
            )
        
        if xscale == 'log':
            if xbreaks is None:
                xbreaks = [input_titration.min(), 1, input_titration.max()]
                xlabels = [f'{int(np.log10(i))}' for i in xbreaks]
            p = p + scale_x_log10(breaks = xbreaks,
                                 labels = xlabels)
            p = p + xlab('$log_{10}[M1]_{tot}$')
        elif xscale == 'linear':
            if xbreaks is None:
                xbreaks = np.linspace(input_titration.min(), input_titration.max(), 3)
                xlabels = xbreaks
            p = p + scale_x_continuous(breaks=xbreaks, 
                                       labels = xlabels) + theme(axis_text_x = element_text(angle = 45))
            p = p + xlab('$[M1]_{tot}$')
        else:
            raise ValueError('Unexpected value for xscale. Please choose "log" or "linear"')
        
        if wadjust is not None:
            p = p + theme(subplots_adjust={'wspace': wadjust})
            
        plot_list[i] = p
        out_df_list[i] = out_df
        
        if save:
            p.save(f'{outfileprefix}{"_".join(out_name)}eq_titrationCurves_univXparam{m}M_{norm}Norm_{xscale}Xscale.pdf', dpi=300)

    return plot_list, out_df_list


Randomly select 10 different simulations (out of the 100) to plot. 

In [16]:
rng = np.random.default_rng(42)
univ_set10 = rng.choice(100, size = 10, replace = False)
np.save(f'{outfileprefix}plotted_univ_set_titratedParam.npy', univ_set10)

In [17]:
_, _ = plot_curves_facet_univXparam('../data/20220420_1input_randomParams/S_all_3M_100k_paramTitration.npy', 
                                    input_titration, m = 3, n_input=1,
                                    univset = univ_set10, xscale = 'log',
                                    norm = 'max', save = True, outfileprefix=outfileprefix)

In [41]:
_, _ = plot_curves_facet_univXparam('../data/20220420_1input_randomParams/S_all_4M_100k_paramTitration.npy', 
                                    input_titration, m = 4, n_input=1,
                                    univset = univ_set10, xscale = 'log',
                                    norm = 'max', fontsize=8,  save = True, outfileprefix=outfileprefix)

In [42]:
_, _ = plot_curves_facet_univXparam('../data/20220420_1input_randomParams/S_all_5M_100k_paramTitration.npy', 
                                    input_titration, m = 5, n_input=1,
                                    univset = univ_set10, xscale = 'log',
                                    norm = 'max', fontsize=6, linesize = 0.25,
                                    save = True, outfileprefix=outfileprefix)

In [43]:
_, _ = plot_curves_facet_univXparam('../data/20220420_1input_randomParams/S_all_6M_100k_paramTitration.npy', 
                                    input_titration, m = 6, n_input=1,
                                    univset = univ_set10, xscale = 'log',
                                    norm = 'max', fontsize=4, linesize = 0.25,
                                    save = True, outfileprefix=outfileprefix)

Write function to plot all dimer output curves in the same figure. Columns correspond to different dimers and rows correspond to different parameter universes

In [30]:
def plot_curves_dimerXuniv(simfile, input_titration, m = 3, 
                           facet = 'univ~species', n_univ = 100, xscale = 'log',
                           univset = None, nrow = 10, norm = 'max', 
                           palette = None, fontsize = 12, linesize = 0.5, wadjust = None, 
                           save = False, outfileprefix = ''):
    """
    Plot output curves for 1-input dimer network simulations.
    
    Parameters
    ----------
    simfile : string
        path to .npy file containing results of the network simulation
        Loaded into S_all
    input_titration : array_like. shape (n_titrations,)
        initial concentration of the input monomer species. 
    m : int. Default 3
        Number of monomer species. Default is 3. 
    facet : string. Options 'univ' or 'univ~species'. Default 'univ~species'
        indicates how to facet the plots in the figure. 
        If 'univ' then each suplot shows the output curves for all species for a given parameter universe
        If 'univ~species' then each suplot shows the output curve for a single species for a given parameter universe
        with the different species arranged along columns. 
    n_univ : int. Default 100
        Number of parameter universes to plot in the figure. 
        Each parameter universes corresponds to a row in the figure. 
        Should be ≤ S_all.shape[2]
    xscale : string. Default 'log'
        scale for x-axis. Should be 'log' or 'linear'
    univset : array_like. shape (n_univ, ). Default None
        index of the parameter universes to plot. 
        if None then univset = np.arange(n_univ)
    nrow : int. Default 10
        Number rows for facetting the plots. 
    norm : string. Default 'max'. Options are 'max', 'scale' or 'none'
        Indicates how to normalize the output concentrations.
    palette : None or list of hexcode colors. Default None. 
        Color palette for output curves used if facet = 'univ'. 
    fontsize : int. Default 12
        font size for text
    linesize : int. Default 0.5
        size for curve thickness. 
    wadjust : float. Default 0.25
        horizontal spacing between subplots.  
    save : Bool. Default False
        If True, save plot figure
    outfileprefix : string. Default ''
        relative path for saving figure. 
    Returns
    -------
    p : plotnine figure object. 
    """
    S_all = np.load(simfile)
    species_names = np.array(make_nXn_species_names(m))
    dimer_names = species_names[m:]
    
    if univset is None:
        univset = np.arange(n_univ)
    else:
        n_univ = len(univset)
        
    S_subset = S_all[:,:,univset]
    S_subset_flat = np.moveaxis(S_subset, 2, 0).reshape(len(input_titration)*n_univ,len(species_names))
    
    S_subset_flat_df = pd.DataFrame(S_subset_flat[:,m:], columns=dimer_names)
    S_subset_flat_df['univ'] = np.repeat(univ_set, len(input_titration))
    S_subset_flat_df['x'] = np.tile(input_titration, n_univ)
    
    S_subset_flat_df_long = S_subset_flat_df.melt(id_vars=['x','univ'], var_name='species', value_name='out')
    
    if norm == 'max':
        S_subset_flat_df_long['out_norm'] = S_subset_flat_df_long.groupby(['univ', 'species'], group_keys=False).apply(lambda x: x.out/x.out.max())
    elif norm == 'scale':
        S_subset_flat_df_long['out_norm'] = S_subset_flat_df_long.groupby(['univ', 'species'], group_keys=False).apply(lambda x: x.out-x.out.min())
        S_subset_flat_df_long['out_norm'] = S_subset_flat_df_long.groupby(['univ', 'species'], group_keys=False).apply(lambda x: x.out_norm/x.out_norm.max())
    elif norm == 'none':
        S_subset_flat_df_long['out_norm'] = S_subset_flat_df_long.out
    else:
        raise ValueError('Unexpected value for norm. Please choose from ["max", "scale", "none"]')
    
    if facet == 'univ':
        plottype = 'overlay'
        p = (ggplot(S_subset_flat_df_long)
         + geom_line(aes(x = 'x', y = 'out_norm', color = 'species'), size = linesize)
         + facet_wrap('~univ', nrow = nrow)
         + theme_classic()
         + ylab('Normalized equil. concentration')
         + scale_y_continuous(breaks = [0,1])
         + theme(strip_background=element_blank(), 
             panel_border=element_blank(),
             axis_line = element_line(size =0.5),
             axis_ticks = element_line(size = 0.5),
             axis_ticks_minor_x=element_blank(),
             aspect_ratio=0.7,
             text=element_text(family='Helvetica', color='black', size = fontsize))
        )
        if palette is not None:
            p = p+scale_color_manual(palette)
            
    elif facet == 'univ~species':
        plottype = 'facet'
        univset = univset[:nrow]
        n_univ = nrow
        p = (ggplot(S_subset_flat_df_long.query('univ in @univset'))
         + geom_line(aes(x = 'x', y = 'out_norm'), size = linesize)
         + facet_grid('univ~species')
         + theme_classic()
         + ylab('Normalized equil. concentration')
         + scale_y_continuous(breaks = [0,1])
         + theme(strip_background=element_blank(), 
             panel_border=element_blank(),
             axis_line = element_line(size =0.5),
             axis_ticks = element_line(size = 0.5),
             axis_ticks_minor_x=element_blank(),
             aspect_ratio=0.7,
             text=element_text(family='Helvetica', color='black', size = fontsize))
        )
    else:
        raise ValueError('Unexpected value for norm. Please choose either "univ" or "univ~species"')
        
    if xscale == 'log':
        maxmin1 = [input_titration.min(), 1, input_titration.max()]
        p = p + scale_x_log10(breaks = maxmin1,
                             labels = [f'{int(np.log10(i))}' for i in maxmin1])
        p = p + xlab('$log_{10}[M1]_{tot}$')
    elif xscale == 'linear':
        maxminmid = np.linspace(input_titration.min(), input_titration.max(), 3)
        p = p + scale_x_continuous(breaks=maxminmid) + theme(axis_text_x = element_text(angle = 45))
        p = p + xlab('$[M1]_{tot}$')
    else:
        raise ValueError('Unexpected value for xscale. Please choose "log" or "linear"')
    
    if wadjust is not None:
        p = p + theme(subplots_adjust={'wspace': wadjust})
            
    if save:
        p.save(f'{outfileprefix}DimerEq_curves_{plottype}_{n_univ}univ_{m}M_{norm}Norm_{xscale}Xscale.pdf', dpi=300)

    return p


In [32]:


_ = plot_curves_dimerXuniv('../data/20220420_1input_randomParams/S_all_3M_1000k.npy',
                              input_titration, m = 3, facet = 'univ~species', 
                              n_univ = 100, xscale = 'log', univset = univ_set, 
                              norm = 'max', save = True, outfileprefix=outfileprefix)

In [34]:


_ = plot_curves_dimerXuniv('../data/20220420_1input_randomParams/S_all_4M_1000k.npy',
                              input_titration, m = 4, facet = 'univ~species', 
                              n_univ = 100, xscale = 'log', univset = univ_set, 
                              norm = 'max', save = True, outfileprefix=outfileprefix)

In [35]:


_ = plot_curves_dimerXuniv('../data/20220420_1input_randomParams/S_all_5M_1000k.npy',
                              input_titration, m = 5, facet = 'univ~species', 
                              n_univ = 100, xscale = 'log', univset = univ_set, 
                              norm = 'max', save = True, outfileprefix=outfileprefix)

In [36]:


_ = plot_curves_dimerXuniv('../data/20220420_1input_randomParams/S_all_6M_1000k.npy',
                              input_titration, m = 6, facet = 'univ~species', 
                              n_univ = 100, xscale = 'log', univset = univ_set, 
                              norm = 'max', save = True, outfileprefix=outfileprefix)

In [46]:
%watermark --iversions
%watermark -p plotnine

pandas    : 1.4.1
sys       : 3.9.7 (default, Sep 16 2021, 08:50:36) 
[Clang 10.0.0 ]
eqtk      : 0.1.2
palettable: 3.3.0
numpy     : 1.20.3
scipy     : 1.7.3
seaborn   : 0.11.2
matplotlib: 3.5.1

plotnine: 0.8.0

